# Database Setup and Seeding Notebook

This notebook handles database initialization, table creation, and seeding with initial data.

## Overview
The notebook performs the following operations:
1. Import required modules and load configuration
2. Create database tables
3. Seed initial users (admin, test users)
4. Seed initial services (Quillbot, Grammarly, ChatGPT)
5. Verify the setup

## Prerequisites
- Ensure your `.env` file is properly configured
- Database connection should be working
- All required packages should be installed

In [ ]:
# Import required modules
import sys
import os
from pathlib import Path
sys.path.append(os.path.dirname(os.path.abspath('.')))

from core.security import get_password_hash
from core.config import settings
from db.session import Base, engine
from db.models.user import User
from db.models.service import Service
from sqlalchemy.orm import Session
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print("✓ All modules imported successfully")
print(f"Database URL: {settings.DATABASE_URL}")
print(f"Admin Username: {settings.ADMIN_USERNAME}")
print(f"Admin Email: {settings.ADMIN_EMAIL}")

## 1. Create Database Tables
First, let's create all the necessary database tables.

In [ ]:
print("Creating database tables...")

try:
    # Create all tables
    Base.metadata.create_all(bind=engine)
    print("✓ All database tables created successfully")
except Exception as e:
    print(f"✗ Error creating tables: {e}")
    raise e

## 2. Define Sample Data
Define the sample data that will be seeded into the database.

In [ ]:
# Sample users data
SAMPLE_USERS = {
    "admin": {
        "username": settings.ADMIN_USERNAME,
        "email": settings.ADMIN_EMAIL,
        "user_id": settings.ADMIN_USER_ID,
        "hashed_password": get_password_hash(settings.ADMIN_PASSWORD),
        "role": settings.ADMIN_ROLE,
        "services": [
            {"service_id": "qb1", "end_date": "31/12/2025", "is_active": True, "credits": 3500},  # 1 year Quillbot
            {"service_id": "gram1", "end_date": "31/12/2025", "is_active": True, "credits": 3000},  # 1 year Grammarly
            {"service_id": "chat1", "end_date": "31/10/2025", "is_active": True, "credits": 4500}   # 1 year ChatGPT
        ],
        "credits": 100000,
        "btc_address": "bc1qxy2kgdygjrsqtzq2n0yrf2493p83kkfjhx0wlh",
        "profile": {
            "first_name": "Admin",
            "last_name": "User",
            "phone": "+1-555-0123",
            "country": "United States",
            "timezone": "UTC-5",
            "preferences": {
                "email_notifications": True,
                "sms_notifications": False,
                "theme": "dark"
            }
        }
    },
    "testuser": {
        "username": "testuser",
        "email": "test@example.com",
        "user_id": "testuser",
        "hashed_password": get_password_hash("userpass123"),
        "role": "user",
        "services": [
            {"service_id": "qb1", "end_date": "31/12/2025", "is_active": True, "credits": 500}  # 1 month Quillbot
        ],
        "credits": 500,
        "btc_address": "btc-testuser",
        "profile": {
            "first_name": "Test",
            "last_name": "User",
            "phone": "+1-555-9999",
            "country": "Canada",
            "timezone": "UTC-8",
            "preferences": {
                "email_notifications": True,
                "sms_notifications": True,
                "theme": "light"
            }
        }
    },
    "premiumuser": {
        "username": "premiumuser",
        "email": "premium@example.com",
        "user_id": "premiumuser",
        "hashed_password": get_password_hash("premium123"),
        "role": "user",
        "services": [
            {"service_id": "qb2", "end_date": "30/11/2025", "is_active": True, "credits": 1200},  # 3 months Quillbot
            {"service_id": "chat1", "end_date": "31/10/2025", "is_active": True, "credits": 1500}   # 3 months ChatGPT
        ],
        "credits": 2500,
        "btc_address": "bc1qpremiumuser123456789",
        "profile": {
            "first_name": "Premium",
            "last_name": "Customer",
            "phone": "+1-555-8888",
            "country": "United Kingdom",
            "timezone": "UTC+0",
            "preferences": {
                "email_notifications": True,
                "sms_notifications": False,
                "theme": "auto"
            }
        }
    }
}

# Sample services data
SAMPLE_SERVICES = {
    "Quillbot": {
        "name": "Quillbot",
        "image": "https://via.placeholder.com/300x200/4F46E5/FFFFFF?text=Quillbot",
        "accounts": [
            {"id": "qb1", "password": "pass1", "end_date": "31/12/2025", "is_active": True},
            {"id": "qb2", "password": "pass2", "end_date": "30/11/2025", "is_active": True}
        ]
    },
    "Grammarly": {
        "name": "Grammarly", 
        "image": "https://via.placeholder.com/300x200/10B981/FFFFFF?text=Grammarly",
        "accounts": [
            {"id": "gram1", "password": "pass3", "end_date": "31/12/2025", "is_active": True}
        ]
    },
    "ChatGPT": {
        "name": "ChatGPT",
        "image": "https://via.placeholder.com/300x200/8B5CF6/FFFFFF?text=ChatGPT", 
        "accounts": [
            {"id": "chat1", "password": "pass4", "end_date": "31/01/2026", "is_active": True}
        ]
    }
}

print("✓ Sample data defined successfully")
print(f"Users to create: {list(SAMPLE_USERS.keys())}")
print(f"Services to create: {list(SAMPLE_SERVICES.keys())}")

## 3. Seed Users
Seed the database with initial user data if no users exist.

In [ ]:
print("Seeding users...")

try:
    with Session(bind=engine, future=True) as db:
        # Check if users already exist
        existing_users = db.query(User).count()
        print(f"Found {existing_users} existing users")
        
        if existing_users == 0:
            print("No users found, creating sample users...")
            
            for username, user_data in SAMPLE_USERS.items():
                user = User(
                    user_id=user_data.get("user_id", username),
                    username=user_data["username"],
                    email=user_data["email"],
                    hashed_password=user_data["hashed_password"],
                    role=user_data.get("role", "user"),
                    credits=user_data.get("credits", 0),
                    btc_address=user_data.get("btc_address", ""),
                    services=user_data.get("services", []),
                    profile=user_data.get("profile", {}),
                )
                db.add(user)
                print(f"  ✓ Added user: {username}")
            
            db.commit()
            print("✓ All users created successfully")
        else:
            print("Users already exist, skipping user creation")
            
except Exception as e:
    print(f"✗ Error seeding users: {e}")
    raise e

## 4. Seed Services
Seed the database with initial service data if no services exist.

In [ ]:
print("Seeding services...")

try:
    with Session(bind=engine, future=True) as db:
        # Check if services already exist
        existing_services = db.query(Service).count()
        print(f"Found {existing_services} existing services")
        
        if existing_services == 0:
            print("No services found, creating sample services...")
            
            for service_name, service_data in SAMPLE_SERVICES.items():
                service = Service(
                    name=service_data["name"],
                    image=service_data.get("image", ""),
                    accounts=service_data.get("accounts", []),
                    is_active=True,
                )
                db.add(service)
                print(f"  ✓ Added service: {service_name}")
            
            db.commit()
            print("✓ All services created successfully")
        else:
            print("Services already exist, skipping service creation")
            
except Exception as e:
    print(f"✗ Error seeding services: {e}")
    raise e

## 5. Verify Setup
Verify that the database setup was successful by checking the created data.

In [ ]:
print("Verifying database setup...")

try:
    with Session(bind=engine, future=True) as db:
        # Count users and services
        user_count = db.query(User).count()
        service_count = db.query(Service).count()
        
        print(f"✓ Database verification complete:")
        print(f"  - Users: {user_count}")
        print(f"  - Services: {service_count}")
        
        # Show some sample data
        if user_count > 0:
            print("\nSample users:")
            users = db.query(User).limit(3).all()
            for user in users:
                print(f"  - {user.username} ({user.role}): {user.credits} credits")
        
        if service_count > 0:
            print("\nSample services:")
            services = db.query(Service).limit(3).all()
            for service in services:
                print(f"  - {service.name}: {len(service.accounts or [])} accounts")
        
        print("\n✅ Database setup completed successfully!")
        
except Exception as e:
    print(f"✗ Error verifying setup: {e}")
    raise e

## 6. Optional: Reset Database
If you need to reset the database, uncomment and run the cell below.
**Warning: This will delete all data!**

In [ ]:
# UNCOMMENT ONLY IF YOU WANT TO RESET THE DATABASE
# print("⚠️  WARNING: This will delete all data!")
# 
# try:
#     # Drop all tables
#     Base.metadata.drop_all(bind=engine)
#     print("✓ All tables dropped")
#     
#     # Recreate tables
#     Base.metadata.create_all(bind=engine)
#     print("✓ Tables recreated")
#     
#     print("\nNow you can run the seeding cells above to recreate the data.")
#     
# except Exception as e:
#     print(f"✗ Error resetting database: {e}")
#     raise e